### Import library

In [1]:
import os
os.chdir('../..')
os.getcwd()

'C:\\Users\\csia7\\OneDrive\\문서\\GitHub\\WQBrain_2024_API'

In [2]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import pygwalker as pyg
import glob
import re

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [3]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_3SgKeUmnaY358hvDYYtMi2Rwv9Tp

 


In [16]:
ace.check_session_timeout(s)

7285.22793

## Alpha Mixing Templates

## Read all passed alphas

In [8]:
os.chdir('Results/signal')
file_direc = glob.glob('*')
file_direc
os.chdir('../..')

['A_signalform.csv',
 'B_signalform.csv',
 'C_signalform.csv',
 'D_signalform.csv',
 'pre_signalform.csv',
 'Sep03GLB_signalform.csv']

In [9]:
expressions = {}
for file in file_direc:
    expressions[file.split('_')[0]] = pd.read_csv(file)['expression']
#expressions

In [10]:
expressions.keys()

dict_keys(['A', 'B', 'C', 'D', 'pre', 'Sep03GLB'])

In [9]:
expressions['pre']

0    sig_pre_0 = group_zscore(ts_zscore(ts_backfill...
Name: expression, dtype: object

## Template A & pre mixing
use top 32 results

In [59]:
#region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01

In [11]:
A = expressions['A'][:32]
pre = expressions['pre']
#Sep03GLB = expressions['Sep03GLB']

In [17]:
alpha_list_Apre = []

for i in range(5):
    alpha_list_Apre += [ace.generate_alpha(f'{A[i]};{pre[0]};add(scale_down(signal_A_{i}), scale_down(signal_pre_{0}), filter = false)', region= "GLB", universe = "MINVOL1M", neutralization = x, truncation = 0.01, decay = 5) for x in ['COUNTRY', 'MARKET']]

alpha_list_Apre

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 5,
   'neutralization': 'COUNTRY',
   'truncation': 0.01,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21);factor_A_0 = group_zscore(ts_ir(returns, 252), industry);alpha_A_0 = regression_neut(stock_A_0, factor_A_0);signal_A_0 = trade_when(pro_A_0<=0.5, alpha_A_0, pro_A_0>0.5);sig_pre_0 = group_zscore(ts_zscore(ts_backfill(anl69_epss_best_eeps_cur_yr, 21), 21), industry);factor_pre_0 = group_zscore(ts_ir(returns, 252), industry);value_pre_0 = floor(rank(fnd28_value_05480/close)*4.99);signal_pre_0 = group_neutralize(sig_pre_0, value_pre_0 *factor_pre_0);add(scale_down(signal_A_0), scale_down(signal_pre_0), filter = false)'},
 {'type

In [18]:
result = ace.simulate_alpha_list_multi(s, alpha_list_Apre)

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [19:56<00:00, 299.11s/it]


In [19]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,...,self_correlation,prod_correlation,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,14902127,20000000,1008,970,0.3170,0.1439,0.0310,0.000908,2.51,3.72,...,NaN,NaN,pro_A_3 = oth460_1l_acda;stock_A_3 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
1,14551983,20000000,1007,971,0.3184,0.1405,0.0294,0.000883,2.46,3.70,...,NaN,NaN,pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
2,18618818,20000000,1012,968,0.2972,0.1798,0.0693,0.001210,2.23,2.87,...,NaN,NaN,pro_A_3 = oth460_1l_acda;stock_A_3 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
3,16207450,20000000,736,704,0.4336,0.1565,0.0292,0.000722,2.18,3.63,...,NaN,NaN,pro_A_4 = oth460_1l_m3r_mtn_spb_se;stock_A_4 =...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
4,18236224,20000000,1010,969,0.2989,0.1761,0.0705,0.001178,2.18,2.84,...,NaN,NaN,pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
5,20918798,20000000,739,703,0.4072,0.2020,0.0642,0.000992,2.16,3.07,...,0.7839,0.7839,pro_A_4 = oth460_1l_m3r_mtn_spb_se;stock_A_4 =...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
6,18571304,20000000,596,580,0.3217,0.1793,0.0752,0.001115,1.98,2.65,...,NaN,NaN,pro_A_2 = oth460_1l_ccns;stock_A_2 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
7,18201387,20000000,595,580,0.3233,0.1758,0.0758,0.001087,1.93,2.62,...,NaN,NaN,pro_A_1 = oth460_1l_ccns;stock_A_1 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
8,14696679,20000000,593,580,0.3495,0.1419,0.0378,0.000812,1.89,2.97,...,NaN,NaN,pro_A_2 = oth460_1l_ccns;stock_A_2 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
9,14354978,20000000,593,581,0.3509,0.1386,0.0375,0.000790,1.84,2.93,...,0.6440,0.6440,pro_A_1 = oth460_1l_ccns;stock_A_1 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS


In [20]:
path = 'Results/mixing'
result_st1.to_csv(path + '/A&pre_mixing_add.csv')

In [21]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df.head()

,alpha_id,endDate,limit,name,result,startDate,themes,value,year
0,5wXxbjz,NaN,1.58,LOW_SHARPE,PASS,NaN,NaN,2.6200,NaN
1,5wXxbjz,NaN,1.00,LOW_FITNESS,PASS,NaN,NaN,1.9300,NaN
2,5wXxbjz,NaN,0.01,LOW_TURNOVER,PASS,NaN,NaN,0.3233,NaN
3,5wXxbjz,NaN,0.70,HIGH_TURNOVER,PASS,NaN,NaN,0.3233,NaN
4,5wXxbjz,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,NaN


In [22]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:[]
Passed alphas:['lVmdbw5', 'qAR817v', '5wXxbjz', '7wpWEOO', 'qAR8oeA', 'Qa8MLpM', 'jZ7vbKk', 'VpWVgnb', 'jZ7vNzj', '17Me9Nm']


In [25]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['Sep05_mixing'])

In [24]:
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas}
submit_result

{'lVmdbw5': False,
 'qAR817v': False,
 '5wXxbjz': False,
 '7wpWEOO': False,
 'qAR8oeA': False,
 'Qa8MLpM': False,
 'jZ7vbKk': False,
 'VpWVgnb': False,
 'jZ7vNzj': False,
 '17Me9Nm': False}

### Library Fuctions.

following are some other functions that you can use for your own analysis

**get_alpha_pnl(s, alpha_id)** - to get the pnl for an alpha

**get_alpha_yearly_stats(s, alpha_id)** - to get yearly statistics for an alpha

**get_self_corr(s, alpha_id)** - to get self correlation results for an alpha

**get_prod_corr(s, alpha_id)** - to get prod correlation results for an alpha

**get_check_submission(s, alpha_id)** - to get check submission result for an alpha

**check_self_corr_test(s, alpha_id)** - to check if alpha passes self correlation test (self_corr<0.7)

**check_prod_corr_test(s, alpha_id)** - to check if alpha passes prod correlation test (prod_corr<0.7)

**perfomance_comparison(s, alpha_id)** - to get the result of performance comparison for an alpha merged performance

In [37]:
ace.get_alpha_pnl(s, result[0]['alpha_id'])

,Pnl,alpha_id
Date,,
2012-01-23,0.0,dx9ALqJ
2012-01-24,52190.0,dx9ALqJ
2012-01-25,32038.0,dx9ALqJ
2012-01-26,37403.0,dx9ALqJ
2012-01-27,53631.0,dx9ALqJ
...,...,...
2022-01-17,12935472.0,dx9ALqJ
2022-01-18,12935661.0,dx9ALqJ
2022-01-19,12967474.0,dx9ALqJ
